In [45]:
import subprocess
import imageio
import numpy as np
import matplotlib.pyplot as plt
import os
import time

In [8]:
def read_grids_from_file(grid_file):
    with open(grid_file, 'r') as file:
        lines = file.readlines()

    grids = []
    current_grid = []
    for line in lines:
        if line.strip() == "":
            if current_grid:
                grids.append(np.array(current_grid))
                current_grid = []
        else:
            current_grid.append(list(map(int, line.strip().split())))

    if current_grid:
        grids.append(np.array(current_grid))

    return grids

In [32]:
def visualize_grid(grid, output_file):
    fig, ax = plt.subplots()
    cmap = plt.cm.viridis
    cmap.set_bad(color='black')
    masked_grid = np.ma.masked_where(grid == -1, grid)
    
    ax.matshow(masked_grid, cmap=cmap, interpolation='nearest')
    
    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            if grid[i, j] != -1:
                ax.text(j, i, str(grid[i, j]), va='center', ha='center', color='white')
            else:
                ax.add_patch(plt.Rectangle((j - 0.5, i - 0.5), 1, 1, fill=True, color='black'))

    plt.savefig(output_file)
    plt.close()

In [13]:
def create_gif_from_grids(grids, gif_file, temp_dir='temp_frames', duration=0.5):
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)

    frame_files = []
    for i, grid in enumerate(grids):
        frame_file = os.path.join(temp_dir, f'frame_{i:04d}.png')
        visualize_grid(grid, frame_file)
        frame_files.append(frame_file)

    with imageio.get_writer(gif_file, mode='I', duration=duration) as writer:
        for frame_file in frame_files:
            image = imageio.imread(frame_file)
            writer.append_data(image)

    for frame_file in frame_files:
        os.remove(frame_file)
    os.rmdir(temp_dir)

In [29]:
def clear_file(file_path):
    with open(file_path, 'w') as file:
        pass  # Opening in 'w' mode will truncate the file

In [52]:
def run_command(command):
    try:
        result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print("Command output:", result.stdout.decode())
        print("Command error (if any):", result.stderr.decode())
    except subprocess.CalledProcessError as e:
        print("Error executing command:", e)
        print("Command output:", e.stdout.decode())
        print("Command error:", e.stderr.decode())

In [61]:
command = "g++ -O3 -o main main.cpp"
run_command(command)

Command output: 
Command error (if any): main.cpp:82:8: warning: 'auto' type specifier is a C++11 extension [-Wc++11-extensions]
  for (auto cell : nets[net]) {
       ^
main.cpp:82:18: warning: range-based for loop is a C++11 extension [-Wc++11-extensions]
  for (auto cell : nets[net]) {
                 ^
main.cpp:130:16: warning: 'auto' type specifier is a C++11 extension [-Wc++11-extensions]
    for (const auto& row : grid) {
               ^
main.cpp:130:26: warning: range-based for loop is a C++11 extension [-Wc++11-extensions]
    for (const auto& row : grid) {
                         ^
main.cpp:131:20: warning: 'auto' type specifier is a C++11 extension [-Wc++11-extensions]
        for (const auto& cell : row) {
                   ^
main.cpp:131:31: warning: range-based for loop is a C++11 extension [-Wc++11-extensions]
        for (const auto& cell : row) {
                              ^
main.cpp:182:14: warning: 'auto' type specifier is a C++11 extension [-Wc++11-extensions

In [62]:
run_command("./main")
time.sleep(2)
grid_file = 'grid.txt'  # Path to the grid updates text file
gif_file = 'grid_evolution.gif'  # Output GIF file name

grids = read_grids_from_file(grid_file)
create_gif_from_grids(grids, gif_file)
clear_file(grid_file)

Command output: Initial Grid (binary): 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 
1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
Initial Grid (actual): 
250 255 311 180 113 369 305 166 158 97 51 190 240 82 217 285 243 -- 349 218 224 330 309 

/var/folders/rj/rk3nbk3537g9r7jt8g9_3chw0000gn/T/ipykernel_2421/3938710260.py:13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(frame_file)
